In [1]:
import os
import cv2

In [2]:
input_dir = r'data\train\masks'
output_dir = r'data\train\labels'

In [16]:
os.listdir(input_dir)[:1][0]

'15008_2_mask.png'

In [17]:
image_path = os.path.join(input_dir, os.listdir(input_dir)[:1][0])

mask = cv2.imread(image_path)

cv2.imshow('mask', mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
os.listdir(input_dir)[0]

'15008_2_mask.png'

In [7]:
mask

In [13]:
os.listdir(input_dir)[:1]

['15008_2_mask.png']

In [25]:
for j in os.listdir(input_dir)[:5]:
    image_path = os.path.join(input_dir, j)
    print(input_dir, j)
    # load the binary mask and get its contours
    mask = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)

    H, W = mask.shape
    print(H, W)
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # convert the contours to polygons
    polygons = []
    for cnt in contours:
        if cv2.contourArea(cnt) > 200:
            polygon = []
            for point in cnt:
                x, y = point[0]
                polygon.append(x / W)
                polygon.append(y / H)
            polygons.append(polygon)

    # print the polygons
    with open('{}.txt'.format(os.path.join(output_dir, j)[:-4]), 'w') as f:
        for polygon in polygons:
            for p_, p in enumerate(polygon):
                if p_ == len(polygon) - 1:
                    f.write('{}\n'.format(p))
                elif p_ == 0:
                    f.write('0 {} '.format(p))
                else:
                    f.write('{} '.format(p))

        f.close()

data\train\masks 15008_2_mask.png
512 512
data\train\masks 15020_0_mask.png
512 512
data\train\masks 15039_0_mask.png
512 512
data\train\masks 15069_1_mask.png
512 512
data\train\masks 15085_1_mask.png
512 512


In [32]:
import os
import cv2

input_dir = r'data\train\masks'
output_dir = r'data\train\labels'

os.makedirs(output_dir, exist_ok=True)

for j in os.listdir(input_dir)[:5]:
    image_path = os.path.join(input_dir, j)

    mask = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)

    H, W = mask.shape

    contours, hierarchy = cv2.findContours(mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

    # Initialize list for polygons
    polygons = []

    # Iterate over the contours
    for idx, cnt in enumerate(contours):
        # Process only significant contours
        if cv2.contourArea(cnt) > 200:
            polygon = []
            # Normalize coordinates and add to polygon list
            for point in cnt:
                x, y = point[0]
                polygon.append(x / W)
                polygon.append(y / H)
            # Append polygon based on its hierarchy (external or internal)
            if hierarchy[0][idx][3] == -1:  # External contour (glasses frame)
                polygons.append(polygon)
            else:  # Internal contour (lenses)
                polygons.append(polygon)

    # Write the polygons to a text file
    output_path = os.path.join(output_dir, f"{os.path.splitext(j)[0]}.txt")
    with open(output_path, 'w') as f:
        for polygon in polygons:
            for p_index, p in enumerate(polygon):
                if p_index == 0:
                    f.write(f'0 {p} ')
                elif p_index == len(polygon) - 1:
                    f.write(f'{p}\n')
                else:
                    f.write(f'{p} ')


Processing data\train\masks\15008_2_mask.png
Image dimensions: 512x512
Processing data\train\masks\15020_0_mask.png
Image dimensions: 512x512
Processing data\train\masks\15039_0_mask.png
Image dimensions: 512x512
Processing data\train\masks\15069_1_mask.png
Image dimensions: 512x512
Processing data\train\masks\15085_1_mask.png
Image dimensions: 512x512


In [3]:
import os

In [11]:
import os
import cv2
import numpy as np

from tqdm.auto import tqdm

In [18]:
mask_dir = r'data2\val\masks'
labels_dir = r'data2\val\labels'
imgs_dir = r'data2\val\images'

In [ ]:
os.makedirs(labels_dir, exist_ok=True)

for mask_name, img_name  in tqdm(zip(os.listdir(mask_dir), os.listdir(imgs_dir))):
    mask_path = os.path.join(mask_dir, mask_name)

    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    _, mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)

    H, W = mask.shape

    contours, hierarchy = cv2.findContours(mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

    external_polygons = []
    internal_polygons = []

    for idx, cnt in enumerate(contours):
        if cv2.contourArea(cnt) > 200:
            polygon = []
            for point in cnt:
                x, y = point[0]
                polygon.append(x / W)
                polygon.append(y / H)
            if hierarchy[0][idx][3] == -1: 
                external_polygons.append(polygon)
            else:  
                internal_polygons.append(polygon)

    output_path = os.path.join(labels_dir, f"{img_name[:-4]}.txt")
    with open(output_path, 'w') as f:
        for polygon in external_polygons:
            f.write('0 ')
            f.write(' '.join(map(str, polygon)))
            f.write('\n')
        for polygon in internal_polygons:
            f.write('1 ')
            f.write(' '.join(map(str, polygon)))
            f.write('\n')


In [18]:
import numpy as np

In [50]:
def visualize_polygons(image_shape, txt_file):
    """
    Visualize polygons on a blank image.

    Parameters:
    - image_shape: tuple, the shape of the output image (height, width)
    - txt_file: str, path to the text file containing polygon data
    - output_image_path: str, path to save the output image with visualized polygons
    """
    H, W = image_shape
    image = np.zeros((H, W, 3), dtype=np.uint8)
    
    with open(txt_file, 'r') as f:
        lines = f.readlines()
    
    for line in lines:
        if line.strip():
            points = list(map(float, line.strip().split()[1:]))
            polygon = [(int(points[i] * W), int(points[i+1] * H)) for i in range(0, len(points), 2)]
            
            print(line.strip().split()[0])
            if line.strip().split()[0] == '0':
                color = (0, 255, 0)
            else:
                color = (255, 0, 0)
            cv2.polylines(image, [np.array(polygon, dtype=np.int32)], isClosed=True, color=color, thickness=2)
    
    cv2.imshow('polygon', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
visualize_polygons((512, 512), r'data\train\labels\15085_1_mask.txt')

In [20]:
import yaml

In [22]:
def create_dataset_yaml(dataset_path, train_dir, val_dir, class_names, output_file):
    """
    Create a dataset.yaml file for YOLO training.
    
    :param dataset_path: Path to the dataset directory.
    :param train_dir: Relative path to the training images directory.
    :param val_dir: Relative path to the validation images directory.
    :param class_names: List of class names.
    :param output_file: Path to the output yaml file.
    """
    data = {
        'path': dataset_path,
        'train': train_dir,
        'val': val_dir,
        'nc': len(class_names),
        'names': class_names
    }
    
    with open(output_file, 'w') as file:
        yaml.dump(data, file, default_flow_style=False)
    print(f"Created {output_file}")

In [24]:
dataset_path = r'/kaggle/input/glasses-segm/data'
train_dir = r'images/train'
val_dir = r'images/val'
class_names = ['glasses', 'lenses'] 
output_file = os.path.join('dataset.yaml')

create_dataset_yaml(dataset_path, train_dir, val_dir, class_names, output_file)

Created dataset.yaml
